In [1]:
#imports to work with...
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from torch.utils.data import DataLoader
import torch
import torchvision
from torchvision import transforms
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import label_binarize

from cl_framework.continual_learning.metrics.metric_evaluator_incdec import MetricEvaluatorIncDec
from cl_framework.utilities.matrix_logger import IncDecLogger
from torchmetrics import Recall

In [2]:
task_csv = '../runs_trainings/decremental_multiclass/normal/seed_0/incdec_rNNMEbbU/error_analysis/task_0_test_error_analysis.csv'
task_data = pd.read_csv(task_csv)
print(task_data)
# this is used later to create the class_to_idx
def kinetics_classes(classes_csv):
    df = pd.read_csv(classes_csv)
    classes_behaviors = {}

    for _, row in df.iterrows():
        class_name = row['Class']
        subcategory = row['Subcategory']
        
        # Check if the class_name is already in the dictionary, if not, create a new entry
        if class_name not in classes_behaviors:
            classes_behaviors[class_name] = []
        
        # Add the subcategory to the corresponding class_name entry in the dictionary
        classes_behaviors[class_name].append(subcategory)

    return classes_behaviors

                                            video_path  prediction  target  \
0    ./Kinetics/Videos/id_7qahQViiGlk_000136_000146...           1       0   
1    ./Kinetics/Videos/id_81ygRCk7Hz8_000000_000010...           4       0   
2    ./Kinetics/Videos/id_825Fw81OywM_000028_000038...           0       0   
3    ./Kinetics/Videos/id_863uabO7S5k_000059_000069...           4       0   
4    ./Kinetics/Videos/id_893XEXlkHeU_000002_000012...           0       0   
..                                                 ...         ...     ...   
365  ./Kinetics/Videos/id_JJP5kAysTXI_000014_000024...           4       4   
366  ./Kinetics/Videos/id_JNy7CjCeDmg_000000_000010...           4       4   
367  ./Kinetics/Videos/id_JOAKjYyi7Pg_000116_000126...           4       4   
368  ./Kinetics/Videos/id_JOB4yy5Jyds_000120_000130...           0       4   
369  ./Kinetics/Videos/id_JQyRW2zUptU_000102_000112...           0       4   

       subcategory      food     phone   smoking   fatigue  sel

In [3]:
#create a mapping between classes - behaviors
folder_csv = '../Kinetics/Info/'
class_csv = os.path.join(folder_csv, 'classes.csv')
classes_behaviors = kinetics_classes(class_csv)

#create a index for each class -- {class: idx}
class_to_idx = {key: i for i, key in enumerate(classes_behaviors.keys())}


In [4]:
predictions_names = ['food', 'phone','smoking','fatigue','selfcare']
targets_names = ['target_food','target_phone','target_smoking','target_fatigue','target_selfcare']

criterion_type = 'multiclass'
#criterion_type = 'multilabel'

predictions_list = []
targets_list = []
for i in range(len(predictions_names)):
    if criterion_type == 'multilabel':
        class_targets = task_data[targets_names[i]].tolist()
        targets_list.append(class_targets)
    class_predictions = task_data[predictions_names[i]].tolist()
    predictions_list.append(class_predictions)
   
predictions = torch.Tensor(predictions_list).permute(1,0)
if criterion_type == 'multilabel':
    targets = torch.Tensor(targets_list).permute(1,0)
else:
    targets = torch.Tensor(task_data['target'])

subcategory = task_data['subcategory']
data_path = task_data['video_path']


In [5]:
all_behaviors_dict = {
    'food': [
        'eating burger', 'eating cake', 'eating carrots', 'eating chips', 'eating doughnuts',
        'eating hotdog', 'eating ice cream', 'eating spaghetti', 'eating watermelon',
        'sucking lolly', 'tasting beer', 'tasting food', 'tasting wine', 'sipping cup'
    ],
    'phone': [
        'texting', 'talking on cell phone', 'looking at phone'
    ],
    'smoking': [
        'smoking', 'smoking hookah', 'smoking pipe'
    ],
    'fatigue': [
        'sleeping', 'yawning', 'headbanging', 'headbutting', 'shaking head'
    ],
    'selfcare': [
        'scrubbing face', 'putting in contact lenses', 'putting on eyeliner', 'putting on foundation',
        'putting on lipstick', 'putting on mascara', 'brushing hair', 'brushing teeth', 'braiding hair',
        'combing hair', 'dyeing eyebrows', 'dyeing hair'
    ]
    }


if criterion_type == 'multilabel':
    binarized_targets = targets
    targets = torch.Tensor(torch.argmax(targets, axis=1))
else:
    binarized_targets = torch.Tensor(label_binarize(targets, classes=[i for i in range(5)]))

In [6]:
metric_eval = MetricEvaluatorIncDec('../random_tries/', num_classes=5, criterion_type=criterion_type, all_behaviors_dict=all_behaviors_dict, class_to_idx=class_to_idx)
metric_eval.update(targets, binarized_targets, predictions, subcategory, data_path)

In [7]:
acc, ap, acc_per_class, mean_ap, map_weighted, precision_per_class, recall_per_class, exact_match, ap_per_subcat, recall_per_subcat = metric_eval.get(verbose=True)



 - task accuracy: 0.5243243243243243
 - task average precision: tensor([0.6371, 0.1971, 0.1325, 0.3346, 0.7209])
 - task acc per class: [0.7162162162162162, 0.8783783783783784, 0.918918918918919, 0.7756756756756756, 0.7594594594594595]
 - task precision per class: tensor([0.6316, 0.2581, 0.0000, 0.2857, 0.6202])
 - task recall per class: tensor([0.6000, 0.2667, 0.0000, 0.4400, 0.6667])
 - task mAP: 0.40442556142807007
 - task weighted mAP: 0.5467880964279175


In [8]:
print(ap_per_subcat)
print(recall_per_subcat)

{'eating burger': tensor(0.1075), 'eating cake': tensor(0.2492), 'eating carrots': tensor(0.1255), 'eating chips': tensor(0.0615), 'eating doughnuts': tensor(0.1944), 'eating hotdog': tensor(0.1697), 'eating ice cream': tensor(0.1299), 'eating spaghetti': tensor(0.1538), 'eating watermelon': tensor(0.3146), 'sucking lolly': tensor(0.0973), 'tasting beer': tensor(0.1124), 'tasting food': tensor(0.1410), 'tasting wine': tensor(0.0949), 'sipping cup': tensor(0.2927), 'texting': tensor(0.1300), 'talking on cell phone': tensor(0.0710), 'looking at phone': tensor(0.0743), 'smoking': tensor(0.0449), 'smoking hookah': tensor(0.0649), 'smoking pipe': tensor(0.0563), 'sleeping': tensor(0.0717), 'yawning': tensor(0.1361), 'headbanging': tensor(0.3075), 'headbutting': tensor(0.1731), 'shaking head': tensor(0.0475), 'scrubbing face': tensor(0.5530), 'putting in contact lenses': tensor(0.3533), 'putting on eyeliner': tensor(0.5247), 'putting on foundation': tensor(0.4476), 'putting on lipstick': ten